Load dependencies

In [ ]:
import numpy as np
import pandas as pd
import os
import sys
import json
import tensorflow as tf
from NN_Predictor import NN
import NN_Trainer
import cobra

In [ ]:
from pathlib import Path
path = Path.cwd()
sys.path.append(path)
file_path = os.path.join(path.parent,'files', 'NN')
output_path = os.path.join(path.parent,'files', 'custom scores')

#Tensorflow complains because the NN is not optimized perfectly
tf.get_logger().setLevel('ERROR')

## Introduction

The purpose of this notebook is to show some examples of using the NN

## Making predictions using the NN

Load in your favourite Neural Network

In [ ]:
NN_MS = NN(path=file_path+'/NN_MS_lite.npz')

In [ ]:
file_path+'/NN_MS_lite.npz'

You can also use modeltype

In [ ]:
NN_MS = NN(modeltype='ModelSEED')

Load in the data, the NN requires a binary list indicating presences of reactions, it is important that this is in the right order. However, you can give it a dictionary and it will put it in the right order for you.

load in the dictionary:

In [ ]:
with open(file_path+'/binary_example.json', 'r') as j:
    binary_example = json.loads(j.read())

and make the prediction

In [ ]:
prediction = NN_MS.predict(input=binary_example)

# Multiple predictions

We can also make predictions for multiple models at the same time, starting with a reaction presence dataframe where the rows are the different reactions and the columns are the model ids:

In [ ]:
#multiple binary reaction lists (csv)
input_path = file_path+'/Sample_reaction_presence.csv'
df = pd.read_csv(input_path, index_col=0)

We can then create a array of prediction scores with the same order of reaction and model ids

In [ ]:
NN_MS.predict(input=df)

Of course usually we dont have a dictionary or binary array but rather a model:

In [ ]:
model_path = os.path.join(path.parent, 'files',  'models', 'E_coli_KTE31_388739.3_r30_removed.sbml')
draft_model = cobra.io.read_sbml_model(model_path) 


We can make a list of the reactions in a model

In [ ]:
list_of_reactions = draft_model.reactions.list_attr('id')
NN_MS.predict(input=list_of_reactions)

or just give the model

In [ ]:
NN_MS.predict(input=draft_model)